In [ ]:
import sys 
sys.path.insert(1, "../")
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import ast

import dask.dataframe as dd
from dask.dataframe import from_pandas

from workloads.util import use_results, use_dataset

%load_ext autoreload
%autoreload 2

In [ ]:
experiment = "stl-yahoo-A4-keys-100-interval-10000-events-200000-queries-200000"

results_dir = use_results(experiment)
dataset_dir = use_dataset(experiment)
print(results_dir)
print(dataset_dir)

In [ ]:
#import wandb
#run = wandb.init(project="ralf-stl", entity="ucb-ralf")
#experiment = 'stl-A4-keys-100-interval-10000-events-200000'
#dataset_artifact = run.use_artifact(f'{experiment}:latest', type='dataset')
#results_artifact = run.use_artifact(f'{experiment}-results:latest', type='results')
#dataset_dir = dataset_artifact.download()
#results_dir = results_artifact.download()
#print(dataset_dir)
#print(results_dir)

In [ ]:
os.listdir(results_dir)

In [ ]:
#queries_df = from_pandas(pd.read_csv(f"{dataset_dir}/queries.csv").head(100000), npartitions=1000)
queries_df = pd.read_csv(f"{dataset_dir}/queries.csv")
events_df = pd.read_csv(f"{dataset_dir}/events.csv")
oracle_df = pd.read_csv(f"{dataset_dir}/oracle_features.csv").dropna()
oracle_df["seasonality"] = oracle_df['seasonality'].dropna().apply(ast.literal_eval)
#oracle_df["seasonality"] = oracle_df['seasonality'].dropna().apply(ast.literal_eval)
# for i in oracle_df.index:
#     try:
#         ast.literal_eval(oracle_df.loc[i]["seasonality"])
#     except Exception as e:
#         print(e)
#         print(oracle_df.loc[i]["seasonality"])
#         break

In [ ]:
oracle_df["seasonality"] = oracle_df['seasonality'].dropna().apply(ast.literal_eval)

In [ ]:
from workloads.util import join_queries_features

In [ ]:
results_df = pd.read_csv(f"{results_dir}/results_workers_4_lifo_window_128_slide_24.csv")

In [ ]:
joined_df = join_queries_features(queries_df, results_df).set_index("query_id")

In [ ]:
queries_df.join(joined_df, on="query_id", lsuffix='_query', rsuffix='_feature')

In [ ]:
joined_df

In [ ]:
queries_df.sort_values(by=["key_id", "timestamp_ms"]).iloc[237]

In [ ]:
queries_df.iloc[237]

## Feature Results for Queries 

In [ ]:
results = {}
for result_file in os.listdir(results_dir): 
    results_df = pd.read_csv(f"{results_dir}/{result_file}")
    print(result_file, "updates:", len(results_df.index))
    results_df["seasonality"] = results_df['seasonality'].apply(ast.literal_eval)
    
    joined_df = join_queries_features(queries_df, results_df).set_index("query_id")
    query_result_df = queries_df.join(joined_df, on="query_id", lsuffix='_query', rsuffix='_feature')
    
    #print(results_df["seasonality"][0][0])

#     #print(key_results_df.filter(lambda x: x["timestamp_ms"].max() < 100))
#     query_result_df = queries_df[queries_df.key_id.isin([3, 8])].apply(
#         lambda q: results_df[(results_df["key_id"] == q["key_id"]) & (results_df["timestamp_ms"] <= q["timestamp_ms"])].set_index("timestamp_ms", drop=False).max(), 
# #         meta={
# #             'key_id': 'int64', 
# #             'trend': 'float64', 
# #             'seasonality': 'object', 
# #             'timestamp_ms': 'int64', 
# #             'processing_time': 'float64', 
# #             'runtime': 'float64', 
# #             'ingest_time': 'float64'
# #         },
#         axis=1
#     )
    
    #print("converting to dask...")
    #results_df = from_pandas(results_df, npartitions=128)
    
    #joined_df = queries_df.join(results_df.set_index("key_id"), on="key_id", how="outer", lsuffix='_query', rsuffix='_feature')
    #query_result_df = joined_df[joined_df["timestamp_ms_query"] >= joined_df["timestamp_ms_feature"]].set_index("timestamp_ms_feature", drop=False).groupby(by=["query_id"]).max()
    
    #print("starting computation...")
    #key_result = query_result_df[query_result_df["key_id"] == 3]
    #key_result.compute()
    #print(key_result)
    #print(query_result_df)
    #query_result_df.compute()
    #print(joined_df.columns)
    #print(result_file, query_result_df.columns)
    
    results[result_file] = query_result_df

## Oracle Feature Results for Queries 

In [ ]:
joined_df = join_queries_features(queries_df, oracle_df).set_index("query_id")

In [ ]:
results["oracle"] = queries_df.join(joined_df, on="query_id", lsuffix='_query', rsuffix='_oracle')

In [ ]:
oracle_predictions = results["oracle"].dropna().apply(lambda x: predict(
        int(x["value_query"]), 
        int(x["trend"]), 
        x["seasonality"], 
        int(x["timestamp_ms_query"]), 
        int(x["timestamp_ms_oracle"]), 
        interval=5
    ), axis=1)

In [ ]:
results["oracle"]["prediction"] = oracle_predictions

# Evaluate Approximated Features 

In [ ]:
from workloads.stl.stl_util import predict, predict_seasonality

### Run predictions 

In [ ]:
for key, df in results.items():
    
    if key == "oracle": continue 

    print(key)
    staleness = df["timestamp_ms_query"] - df["timestamp_ms_feature"]
    print("staleness", staleness.mean())
    
    results[key]["prediction"] = df.dropna().apply(lambda x: predict(
        int(x["value"]), 
        int(x["trend"]), 
        x["seasonality"], 
        int(x["timestamp_ms_query"]), 
        int(x["timestamp_ms_feature"]), 
        interval=5
    ), axis=1)
    print("residual", residuals.abs().mean())
    
    results[key]["prediction_seasonality"] = df.dropna().apply(lambda x: predict_seasonality(
        x["seasonality"], 
        int(x["timestamp_ms_query"]), 
        int(x["timestamp_ms_feature"]), 
        interval=5
    ), axis=1)
    print()

In [ ]:
from sktime.performance_metrics.forecasting import mean_squared_scaled_error
from tqdm import tqdm 

In [ ]:
losses = {}
for key, df in results.items():
    
    if key == "oracle": continue 
    print(key)
    losses[key] = {}
    
    joined_df = results[key].join(results["oracle"], on="query_id", rsuffix='_oracle')
    #print(joined_df.key_id_query.value_counts())
    for key_id in tqdm(range(1, 101, 1)): 
        
        df = joined_df[joined_df["key_id_query"] == key_id].dropna(subset=["prediction", "prediction_oracle", "value"])
         
        if len(df.index) == 0: 
            print(key, key_id, "no results")
            continue 
        
        y_true = df["prediction_oracle"]
        y_pred = df["prediction"]
        value = df["value"]
        
        #y_true = results["oracle"][results["oracle"]["key_id_query"] == key_id]["prediction"]
        #y_pred = results[key][results[key]["key_id_query"] == key_id]["prediction"]
        #value = results[key][results[key]["key_id_query"] == key_id]["value"]
        loss = mean_squared_scaled_error(
            y_true=y_true, y_pred=y_pred, y_train=value
        )
        losses[key][key_id] = loss

In [ ]:
for key in losses.keys(): 
    print(key)
    print(sum(list(losses[key].values())))

In [ ]:
residuals = query_oracle_df.apply(lambda x: predict(
        int(x["value_query"]), 
        int(x["trend"]), 
        x["seasonality"], 
        int(x["timestamp_ms"]), 
        int(x["timestamp_ms"]), 
        interval=5
), axis=1)
print(residuals.abs().mean())


In [ ]:
results["results_workers_4_lifo_window_128_slide_24.csv"]

## Per-key Evaluation 

In [ ]:
results["results_workers_4_lifo_window_128_slide_128.csv"]

In [ ]:
results["oracle"][results["oracle"]["key_id_query"] == 1]

In [ ]:
d = results["results_workers_4_lifo_window_128_slide_24.csv"].join(results["oracle"], on="query_id", rsuffix="_oracle")

In [ ]:
d[d["key_id_query"] == 28]

In [ ]:
d